In [4]:
from glob import glob
from matplotlib import pyplot as plt
import numpy as np
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.io import read_image
from tqdm import tqdm

# Loading the Data

In [5]:
import pygame
import sys

# Initialize pygame
pygame.init()

# Set up the display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple Drawing App")

# Set up the colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Set up the drawing variables
radius = 10
drawing = False

# Main loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Left mouse button
                drawing = True
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                drawing = False
        elif event.type == pygame.MOUSEMOTION:
            if drawing:
                pygame.draw.circle(screen, BLACK, event.pos, radius)

    # Update the display
    pygame.display.flip()
    # Cap the frame rate
    pygame.time.Clock().tick(60)


ModuleNotFoundError: No module named 'pygame'

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%%capture
%%bash
if [ ! -d ./symbols ]; then
  unzip /content/drive/MyDrive/Projects/camculator/symbols.zip;
fi

In [15]:
data_path = "./symbols/"

In [16]:
labels = {
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,

    "w": 10,
    "x": 11,
    "y": 12,
    "z": 13,

    "dot": 14,
    "minus": 15,
    "plus": 16,
    "slash": 17,
}

In [17]:
class MyDataset(Dataset):
  def __init__(self, data_path):
    self.data_path = data_path
    self.glob = glob(self.data_path + "*")


  def __len__(self):
    return len(self.glob)


  def __getitem__(self, idx):
    img_path = self.glob[idx]

    pattern = r'symbols/(.*?)\-'
    symbol = re.findall(pattern, img_path)[0]

    label = labels[symbol]

    img = read_image(img_path) / 255.

    return img, label

In [18]:
data = MyDataset(data_path)

data_size = len(data)
print("number of images:", data_size)

val_size = int(data_size * 0.2)
train_size = int(data_size - val_size)

train_data, val_data = random_split(data, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=train_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=val_size, shuffle=True)

number of images: 27000


# Creating the Model

In [19]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 18)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)

    x = self.conv2(x)
    x = F.relu(x)

    x = F.max_pool2d(x, 2)

    x = self.dropout1(x)

    x = torch.flatten(x, 1)

    x = self.fc1(x)
    x = F.relu(x)

    x = self.dropout2(x)
    x = self.fc2(x)

    output = F.log_softmax(x, dim=1)

    return output



# Train

In [20]:
# settings
lr = 0.01
EPOCHS = 300

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device} ")

Device: cuda 


In [22]:
model = Model().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

loss_fn = torch.nn.CrossEntropyLoss()

In [23]:
def train_one_epoch(model, train_loader):
  model.train(True)

  running_loss = 0.
  last_loss = 0.

  for i, (inputs, labels) in enumerate(train_loader):

      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      outputs = model(inputs)

      loss = loss_fn(outputs, labels)

      loss.backward()

      optimizer.step()

      running_loss += loss.item()

  return model, running_loss / len(train_loader)


def evaluate(model, criterion, val_loader):
  model.eval()

  running_loss = 0.0

  with torch.no_grad():

    for inputs, labels in val_loader:

      inputs, labels = inputs.to(device), labels.to(device)

      outputs = model(inputs)

      loss = criterion(outputs, labels)

      running_loss += loss.item()

  return running_loss / len(val_loader)



In [24]:
epoch_number = 0
best_vloss = 1_000_000.
train_losses = []
val_losses = []


for epoch in tqdm(range(EPOCHS), colour="green"):
    print('\nEPOCH {}:'.format(epoch_number + 1))

    model, train_loss = train_one_epoch(model, train_loader)
    val_loss = evaluate(model, loss_fn, val_loader)

    print(f"\ttrain loss: {train_loss:.4}, val loss: {val_loss:.4}")

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    epoch_number += 1

    # avg_vloss = running_vloss / (i + 1)
    # print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    # writer.add_scalars('Training vs. Validation Loss',
    #                 { 'Training' : avg_loss, 'Validation' : avg_vloss },
    #                 epoch_number + 1)
    # writer.flush()

    # Track best performance, and save the model's state
    # if train_loss < best_vloss:
    #     best_vloss = train_loss
    #     model_path = 'model_{}_{}'.format(epoch_number)
    #     torch.save(model.state_dict(), model_path)

  0%|          | 0/300 [00:00<?, ?it/s]


EPOCH 1:


  0%|          | 1/300 [00:05<29:30,  5.92s/it]

	train loss: 2.894, val loss: 5.044

EPOCH 2:


  1%|          | 2/300 [00:11<27:26,  5.52s/it]

...

	train loss: 0.09581, val loss: 0.1001

EPOCH 77:


 26%|██▌       | 77/300 [05:45<16:39,  4.48s/it]

	train loss: 0.09369, val loss: 0.1001

EPOCH 78:


 26%|██▌       | 77/300 [05:47<16:47,  4.52s/it]


KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses[10:150])
plt.plot(val_losses[10:150], c="orange")
plt.show()